In [1]:
%matplotlib inline

import gym
import itertools
import matplotlib
import numpy as np
import time
import os

In [103]:
def create_shared_noise():
    """
    Create a large array of noise to be shared by all workers. Used 
    for avoiding the communication of the random perturbations delta.
    """

    seed = 12345
    count = 250000000
    noise = np.random.RandomState(seed).randn(count).astype(np.float64)
    return noise


class SharedNoiseTable(object):
    def __init__(self, noise, seed = 11):

        self.rg = np.random.RandomState(seed)
        self.noise = noise
        assert self.noise.dtype == np.float64

    def get(self, i, dim):
        return self.noise[i:i + dim]

    def sample_index(self, dim):
        return self.rg.randint(0, len(self.noise) - dim + 1)

    def get_delta(self, dim):
        idx = self.sample_index(dim)
        return idx, self.get(idx, dim)

In [127]:
class Worker(object):
    """ 
    Object class for parallel rollout generation.
    """

    def __init__(self, env_seed,
                 env_name='',
                 policy_params=None,
                 deltas=None,
                 delta_std=0.02,
                 mode=1
                 ):
        
        # initialize OpenAI environment for each worker
        
        self.env = gym.make(env_name)
        self.env.seed(env_seed)
        self.deltas = SharedNoiseTable(deltas,seed=env_seed + 7)
        
        self.ob_dim = env.observation_space.shape[0]
        self.ac_dim = env.action_space.shape[0]
        print(policy_params)
        self.policy = Policy(policy_params)
        self.delta_std = delta_std
        
    def rollout(self,mode,mean,Sigma):
        ob = env.reset()
        total_ob = [ob]
        total_reward = 0
        for t in itertools.count():
            action = self.policy.get_action(ob,mode,mean,Sigma)
            ob,reward,done,_ = env.step(action)
            total_ob.append(ob)
            total_step = t
            total_reward += reward
            if done:
                break
        return total_reward,total_step,total_ob

    def do_rollout(self, weight, num_rollouts,mode,mean,Sigma):
        rollout_rewards,deltas_idx,rollout_ob  = [],[],[]
        steps = 0

        for i_rollout in range(num_rollouts):

            idx, delta = self.deltas.get_delta(weight.size)

            delta = (self.delta_std * delta).reshape(weight.shape)
            deltas_idx.append(idx)
            # compute reward and number of timesteps used for positive perturbation rollout
            self.policy.update_weights(weight + self.delta_std * delta)
            pos_reward, pos_steps, pos_ob  = self.rollout(mode,mean,Sigma)

            # compute reward and number of timesteps used for negative pertubation rollout
            self.policy.update_weights(weight - self.delta_std * delta)
            neg_reward, neg_steps, neg_ob = self.rollout(mode,mean,Sigma) 
            steps += pos_steps + neg_steps

            rollout_rewards.append([pos_reward, neg_reward])
            rollout_ob.append([pos_ob,neg_ob])

        return {'deltas_idx': deltas_idx, 'rollout_rewards': rollout_rewards,
                "steps" : steps, "rollout_ob": rollout_ob}

In [182]:
class ARSLearner(object):
    """ 
    Object class implementing the ARS algorithm.
    """
    def __init__(self, env_name='HalfCheetah-v2',
                 policy_params=None,
                 l_rate=0.02,
                 num_workers=1,
                 delta_std=0.02, 
                 num_iter=1000,
                 step_size=0.01,
                 seed=123,
                 mode=1
                ):
        # create shared table for storing noise
        print("Creating deltas table.")
        deltas_id = create_shared_noise()
        self.deltas = SharedNoiseTable(deltas_id, seed = seed + 3)
        self.l_rate = l_rate
        self.workers = [Worker(seed + 7 * i,
                               env_name=env_name,
                               policy_params=policy_params,
                               deltas=deltas_id,
                               delta_std=delta_std,
                               mode=1) for i in range(num_workers)]
    def train(self,num_iter,num_rollouts,max_b,mode,policy_params):
        
        weight = np.zeros([policy_params['ob_dim'],policy_params['ac_dim']])
        mean = 0
        Sigma = np.identity(policy_params['ob_dim'])

        for t in range(num_iter):
            t1 = time.time()
            result_list = [worker.do_rollout(weight,num_rollouts,mode,mean,Sigma) for worker in self.workers]
            t2 = time.time()
            print('total time of one step', t2 - t1)           
            print('iter ', t,' done')
            
            #Gather the result

            ob_list = [iob for result in result_list for ob in result['rollout_ob'] for iob in ob]
            ob_array = np.array(ob_list)
            mean = np.mean(ob_array)
            Sigma = np.std(ob_array)
            
            result_dict = result_list[0]
            roll_reward = result_dict['rollout_rewards']
            roll_delidx = result_dict['deltas_idx']
            r_list = zip(roll_reward,roll_delidx)

            r_list = sorted(r_list,key=lambda tup: max(tup[0][0],tup[0][1]))
            r_list = r_list[0:max_b]
            #Update the weight
            reward_list = []
            for roll_r,delta_id in r_list:
                cum_diff = (roll_r[0] - roll_r[1]) * self.deltas.get(delta_id,weight.size)
                reward_list.append(roll_r[0])
                reward_list.append(roll_r[1])
            reward_list = np.array(reward_list)
            weight = weight + self.l_rate / np.std(reward_list) * cum_diff.reshape(weight.shape)
            
        return weight

In [183]:
class Policy(object):
    def __init__(self, policy_params):

        self.ob_dim = policy_params['ob_dim']
        self.ac_dim = policy_params['ac_dim']
        self.weight = np.zeros([self.ac_dim,ob_dim])
    def update_weights(self, new_weight):
        self.weight = new_weight
        return
    def get_action(self,ob,mode,mean,Sigma):
        if mode == 1:
            return np.matmul(self.weight,ob)
        if mode == 2:
            Sigma_diag = np.diag(np.diag(Sigma))
            Sigma_t = np.sqrt(np.linalg.inv(Sigma_diag))
            return np.matmul(np.matmul(self.weight,Sigma_t),(ob - mean))

In [ ]:
#Test the algorithm
env_name = 'HalfCheetah-v2'
env = gym.make(env_name)
ac_dim = env.observation_space.shape[0]
ob_dim = env.action_space.shape[0]
policy_params={'ob_dim':ob_dim,'ac_dim':ac_dim}
l_rate=0.02
num_workers=1
delta_std=0.02
num_iter=1000
step_size=0.01
seed=123
mode=1
num_rollouts = 8
max_b=4
Learner = ARSLearner(env_name,
                     policy_params,
                     l_rate,
                     num_workers,
                     delta_std,
                     num_iter,
                     step_size,
                     seed,
                     mode)

weight = Learner.train(num_iter,num_rollouts,max_b,mode,policy_params,)
policy = Policy(policy_params)
policy.update_weights(weight)

Creating deltas table.
{'ob_dim': 6, 'ac_dim': 17}
total time of one step 2.458125114440918
iter  0  done
total time of one step 2.2124271392822266
iter  1  done
total time of one step 1.886685848236084
iter  2  done
total time of one step 1.8678677082061768
iter  3  done
total time of one step 2.3542778491973877
iter  4  done
total time of one step 1.6897187232971191
iter  5  done
total time of one step 1.9016399383544922
iter  6  done
total time of one step 1.6514859199523926
iter  7  done
total time of one step 2.273731231689453
iter  8  done
total time of one step 2.559812068939209
iter  9  done
total time of one step 2.9243950843811035
iter  10  done
total time of one step 1.7537341117858887
iter  11  done
total time of one step 3.411369800567627
iter  12  done
total time of one step 1.8797550201416016
iter  13  done
total time of one step 2.0629491806030273
iter  14  done
total time of one step 1.8385827541351318
iter  15  done
total time of one step 2.700763702392578
iter  16  d

In [151]:
env_name = 'HalfCheetah-v2'
env = gym.make(env_name)

ob = env.reset

TypeError: 'method' object is not iterable